Run this cell once

In [ ]:
%%shell
set -e
wget -nv https://raw.githubusercontent.com/odow/SESO2023/main/install_colab.sh -O /tmp/install_colab.sh
bash /tmp/install_colab.sh  # Takes ~ 2 minutes

Then refresh the page... and run this cell once

In [ ]:
import Downloads, Pkg
Downloads.download("https://raw.githubusercontent.com/odow/SESO2023/main/Project.toml", "/tmp/Project.toml")
Pkg.activate("/tmp/Project.toml")
Pkg.instantiate()  # Can take ~ 7 minutes

# The knapsack problem example

The purpose of this tutorial is to demonstrate how to formulate and solve a
simple optimization problem.

## Required packages

This tutorial requires the following packages:

In [ ]:
using JuMP
import HiGHS

## Formulation

The [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem)
is a classical optimization problem: given a set of items and a container with
a fixed capacity, choose a subset of items having the greatest combined
value that will fit within the container without exceeding the capacity.

The name of the problem suggests its analogy to packing for a trip,
where the baggage weight limit is the capacity and the goal is to pack the
most profitable combination of belongings.

We can formulate the knapsack problem as the integer linear program:
$$
\begin{aligned}
\max \; & \sum_{i=1}^n c_i x_i      \\
s.t. \; & \sum_{i=1}^n w_i x_i \le C, \\
        & x_i \in \{0,1\},\quad \forall i=1,\ldots,n,
\end{aligned}
$$
where $C$ is the capacity, and there is a choice between $n$ items, with
item $i$ having weight $w_i$, profit $c_i$. Decision variable $x_i$ is
equal to 1 if the item is chosen and 0 if not.

This formulation can be written more compactly as:
$$
\begin{aligned}
\max \; & c^\top x       \\
s.t. \; & w^\top x \le C \\
        & x \text{ binary }.
\end{aligned}
$$

## Data

The data for the problem consists of two vectors (one for the profits and one
for the weights) along with a capacity.

There are five objects:

In [ ]:
n = 5;

For our example, we use a capacity of 10 units:

In [ ]:
capacity = 10.0;

and the profit and cost data:

In [ ]:
profit = [5.0, 3.0, 2.0, 7.0, 4.0];
weight = [2.0, 8.0, 4.0, 2.0, 5.0];

## JuMP formulation

Let's begin constructing the JuMP model for our knapsack problem.

First, we'll create a `Model` object for holding model elements as we
construct each part. We'll also set the solver that will ultimately be called
to solve the model, once it's constructed.

In [ ]:
model = Model(HiGHS.Optimizer)

Next we need the decision variables representing which items are chosen:

In [ ]:
@variable(model, x[1:n], Bin)

We now want to constrain those variables so that their combined
weight is less than or equal to the given capacity:

In [ ]:
@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)

Finally, our objective is to maximize the combined profit of the chosen items:

In [ ]:
@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))

Let's print a human-readable description of the model and check that the model
looks as expected:

In [ ]:
print(model)

We can now solve the optimization problem and inspect the results.

In [ ]:
optimize!(model)
solution_summary(model)

The items chosen are

In [ ]:
items_chosen = [i for i in 1:n if value(x[i]) > 0.5]

## Writing a function

After working interactively, it is good practice to implement your model in a
function.

The function can be used to ensure that the model is given well-defined input
data with validation checks, and that the solution process went as expected.

In [ ]:
function solve_knapsack_problem(;
    profit::Vector{Float64},
    weight::Vector{Float64},
    capacity::Float64,
)
    n = length(weight)
    # The profit and weight vectors must be of equal length.
    @assert length(profit) == n
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n], Bin)
    @objective(model, Max, profit' * x)
    @constraint(model, weight' * x <= capacity)
    optimize!(model)
    @assert termination_status(model) == OPTIMAL
    @assert primal_status(model) == FEASIBLE_POINT
    println("Objective is: ", objective_value(model))
    println("Solution is:")
    for i in 1:n
        print("x[$i] = ", round(Int, value(x[i])))
        println(", c[$i] / w[$i] = ", profit[i] / weight[i])
    end
    chosen_items = [i for i in 1:n if value(x[i]) > 0.5]
    return return chosen_items
end

solve_knapsack_problem(; profit = profit, weight = weight, capacity = capacity)

We observe that the chosen items (1, 4, and 5) have the best
profit to weight ratio in this particular example.

## Next steps

Here are some things to try next:

* Call the function with different data. What happens as the capacity
  increases?
* What happens if the profit and weight vectors are different lengths?
* Instead of creating a binary variable with `Bin`, we could have written
  `@variable(model, 0 <= x[1:n] <= 1, Int)`. Verify that this formulation
  finds the same solution. What happens if we are allowed to take more than
  one of each item?

# The diet problem

The purpose of this tutorial is to demonstrate how to incorporate DataFrames
into a JuMP model. As an example, we use classic [Stigler diet problem](https://en.wikipedia.org/wiki/Stigler_diet).

## Required packages

This tutorial requires the following packages:

In [ ]:
using JuMP
import CSV
import DataFrames
import HiGHS
import Test

## Formulation

We wish to cook a nutritionally balanced meal by choosing the quantity of each
food $f$ to eat from a set of foods $F$ in our kitchen.

Each food $f$ has a cost, $c_f$, as well as a macro-nutrient profile
$a_{m,f}$ for each macro-nutrient $m \in M$.

Because we care about a nutritionally balanced meal, we set some minimum and
maximum limits for each nutrient, which we denote $l_m$ and $u_m$
respectively.

Furthermore, because we are optimizers, we seek the minimum cost solution.

With a little effort, we can formulate our dinner problem as the following
linear program:
$$
\begin{aligned}
\min & \sum\limits_{f \in F} c_f x_f \\
\text{s.t.}\ \ & l_m \le \sum\limits_{f \in F} a_{m,f} x_f \le u_m, && \forall m \in M \\
& x_f \ge 0, && \forall f \in F.
\end{aligned}
$$

In the rest of this tutorial, we will create and solve this problem in JuMP,
and learn what we should cook for dinner.

## Data

First, we need some data for the problem. For this tutorial, we'll write CSV
files to a temporary directory from Julia. If you have existing files, you
could change the filenames to point to them instead.

In [ ]:
dir = mktempdir()

The first file is a list of foods with their macro-nutrient profile:

In [ ]:
food_csv_filename = joinpath(dir, "diet_foods.csv")
open(food_csv_filename, "w") do io
    write(
        io,
        """
        name,cost,calories,protein,fat,sodium
        hamburger,2.49,410,24,26,730
        chicken,2.89,420,32,10,1190
        hot dog,1.50,560,20,32,1800
        fries,1.89,380,4,19,270
        macaroni,2.09,320,12,10,930
        pizza,1.99,320,15,12,820
        salad,2.49,320,31,12,1230
        milk,0.89,100,8,2.5,125
        ice cream,1.59,330,8,10,180
        """,
    )
    return
end
foods = CSV.read(food_csv_filename, DataFrames.DataFrame)

Here, $F$ is `foods.name` and $c_f$ is `foods.cost`. (We're also playing
a bit loose the term "macro-nutrient" by including calories and sodium.)

We also need our minimum and maximum limits:

In [ ]:
nutrient_csv_filename = joinpath(dir, "diet_nutrient.csv")
open(nutrient_csv_filename, "w") do io
    write(
        io,
        """
        nutrient,min,max
        calories,1800,2200
        protein,91,
        fat,0,65
        sodium,0,1779
        """,
    )
    return
end
limits = CSV.read(nutrient_csv_filename, DataFrames.DataFrame)

Protein is missing data for the maximum. Let's fix that using `coalesce`:

In [ ]:
limits.max = coalesce.(limits.max, Inf)
limits

## JuMP formulation

Now we're ready to convert our mathematical formulation into a JuMP model.

First, create a new JuMP model. Since we have a linear program, we'll use
HiGHS as our optimizer:

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

Next, we create a set of decision variables `x`, with one element for each row
in the DataFrame, and each `x` has a lower bound of `0`:

In [ ]:
@variable(model, x[foods.name] >= 0)

To simplify things later on, we store the vector as a new column `x` in the
DataFrame `foods`. Since `x` is a `DenseAxisArray`, we first need to convert
it to an `Array`:

In [ ]:
foods.x = Array(x)

Our objective is to minimize the total cost of purchasing food:

In [ ]:
@objective(model, Min, sum(foods.cost .* foods.x));

For the next component, we need to add a constraint that our total intake of
each component is within the limits contained in the `limits` DataFrame:

In [ ]:
@constraint(
    model,
    [row in eachrow(limits)],
    row.min <= sum(foods[!, row.nutrient] .* foods.x) <= row.max,
);

What does our model look like?

In [ ]:
print(model)

## Solution

Let's optimize and take a look at the solution:

In [ ]:
optimize!(model)
Test.@test primal_status(model) == FEASIBLE_POINT
Test.@test objective_value(model) ≈ 11.8288 atol = 1e-4
solution_summary(model)

We found an optimal solution. Let's see what the optimal solution is:

In [ ]:
for row in eachrow(foods)
    println(row.name, " = ", value(row.x))
end

That's a lot of milk and ice cream, and sadly, we only get `0.6` of a
hamburger.

We can also use the function `Containers.rowtable` to easily convert
the result into a DataFrame:

In [ ]:
table = Containers.rowtable(value, x; header = [:food, :quantity])
solution = DataFrames.DataFrame(table)

This makes it easy to perform analyses our solution:

In [ ]:
filter!(row -> row.quantity > 0.0, solution)

## Problem modification

JuMP makes it easy to take an existing model and modify it by adding extra
constraints. Let's see what happens if we add a constraint that we can buy at
most 6 units of milk or ice cream combined.

In [ ]:
dairy_foods = ["milk", "ice cream"]
is_dairy = map(name -> name in dairy_foods, foods.name)
dairy_constraint = @constraint(model, sum(foods[is_dairy, :x]) <= 6)
optimize!(model)
Test.@test termination_status(model) == INFEASIBLE
Test.@test primal_status(model) == NO_SOLUTION
solution_summary(model)

There exists no feasible solution to our problem. Looks like we're stuck
eating ice cream for dinner.

## Next steps

* You can delete a constraint using `delete(model, dairy_constraint)`. Can you
  add a different constraint to provide a diet with less dairy?
* Some food items (like hamburgers) are discrete. You can use `set_integer`
  to force a variable to take integer values. What happens to the solution if
  you do?

# Rocket Control

**This tutorial was originally contributed by Iain Dunning.**

The purpose of this tutorial is to demonstrate how to setup and solve a
nonlinear optimization problem.

The example is an optimal control problem of a nonlinear rocket.

This tutorial uses the following packages:

In [ ]:
using JuMP
import Ipopt
import Plots

## Overview

Our goal is to maximize the final altitude of a vertically launched rocket.

We can control the thrust of the rocket, and must take account of the rocket
mass, fuel consumption rate, gravity, and aerodynamic drag.

Let us consider the basic description of the model (for the full description,
including parameters for the rocket, see [COPS3](https://www.mcs.anl.gov/~more/cops/cops3.pdf)).

There are three state variables in our model:

* Velocity: $x_v(t)$
* Altitude: $x_h(t)$
* Mass of rocket and remaining fuel, $x_m(t)$

and a single control variable:

* Thrust: $u_t(t)$.

There are three equations that control the dynamics of the rocket:

 * Rate of ascent: $$\frac{d x_h}{dt} = x_v$$
 * Acceleration: $$\frac{d x_v}{dt} = \frac{u_t - D(x_h, x_v)}{x_m} - g(x_h)$$
 * Rate of mass loss: $$\frac{d x_m}{dt} = -\frac{u_t}{c}$$

where drag $D(x_h, x_v)$ is a function of altitude and velocity, gravity
$g(x_h)$ is a function of altitude, and $c$ is a constant.

These forces are defined as:

$$D(x_h, x_v) = D_c \cdot x_v^2 \cdot e^{-h_c \left( \frac{x_h-x_h(0)}{x_h(0)} \right)}$$
and
$$g(x_h) = g_0 \cdot \left( \frac{x_h(0)}{x_h} \right)^2$$

We use a discretized model of time, with a fixed number of time steps, $T$.

Our goal is thus to maximize $x_h(T)$.

## Data

All parameters in this model have been normalized to be dimensionless, and
they are taken from [COPS3](https://www.mcs.anl.gov/~more/cops/cops3.pdf).

In [ ]:
h_0 = 1                      # Initial height
v_0 = 0                      # Initial velocity
m_0 = 1.0                    # Initial mass
m_T = 0.6                    # Final mass
g_0 = 1                      # Gravity at the surface
h_c = 500                    # Used for drag
c = 0.5 * sqrt(g_0 * h_0)    # Thrust-to-fuel mass
D_c = 0.5 * 620 * m_0 / g_0  # Drag scaling
u_t_max = 3.5 * g_0 * m_0    # Maximum thrust
T_max = 0.2                  # Number of seconds
T = 1_000                    # Number of time steps
Δt = 0.2 / T;                # Time per discretized step

## JuMP formulation

First, we create a model and choose an optimizer. Since this is a nonlinear
program, we need to use a nonlinear solver like Ipopt. We cannot use a linear
solver like HiGHS.

In [ ]:
model = Model(Ipopt.Optimizer)
set_silent(model)

Next, we create our state and control variables, which are each indexed by
`t`. It is good practice for nonlinear programs to always provide a starting
solution for each variable.

In [ ]:
@variable(model, x_v[1:T] >= 0, start = v_0)           # Velocity
@variable(model, x_h[1:T] >= 0, start = h_0)           # Height
@variable(model, x_m[1:T] >= m_T, start = m_0)         # Mass
@variable(model, 0 <= u_t[1:T] <= u_t_max, start = 0); # Thrust

We implement boundary conditions by fixing variables to values.

In [ ]:
fix(x_v[1], v_0; force = true)
fix(x_h[1], h_0; force = true)
fix(x_m[1], m_0; force = true)
fix(u_t[T], 0.0; force = true)

The objective is to maximize altitude at end of time of flight.

In [ ]:
@objective(model, Max, x_h[T])

Forces are defined as functions:

In [ ]:
D(x_h, x_v) = D_c * x_v^2 * exp(-h_c * (x_h - h_0) / h_0)
g(x_h) = g_0 * (h_0 / x_h)^2

The dynamical equations are implemented as constraints.

In [ ]:
ddt(x::Vector, t::Int) = (x[t] - x[t-1]) / Δt
@constraint(model, [t in 2:T], ddt(x_h, t) == x_v[t-1])
@constraint(
    model,
    [t in 2:T],
    ddt(x_v, t) == (u_t[t-1] - D(x_h[t-1], x_v[t-1])) / x_m[t-1] - g(x_h[t-1]),
)
@constraint(model, [t in 2:T], ddt(x_m, t) == -u_t[t-1] / c);

Now we optimize the model and check that we found a solution:

In [ ]:
optimize!(model)
solution_summary(model)

Finally, we plot the solution:

In [ ]:
function plot_trajectory(y; kwargs...)
    return Plots.plot(
        (1:T) * Δt,
        value.(y);
        xlabel = "Time (s)",
        legend = false,
        kwargs...,
    )
end

Plots.plot(
    plot_trajectory(x_h; ylabel = "Altitude"),
    plot_trajectory(x_m; ylabel = "Mass"),
    plot_trajectory(x_v; ylabel = "Velocity"),
    plot_trajectory(u_t; ylabel = "Thrust");
    layout = (2, 2),
)

## Next steps

* Experiment with different values for the constants. How does the solution
  change? In particular, what happens if you change `T_max`?
* The dynamical equations use rectangular integration for the right-hand side
  terms. Modify the equations to use the [Trapezoidal rule](https://en.wikipedia.org/wiki/Trapezoidal_rule_(differential_equations))
  instead. (As an example, `x_v[t-1]` would become
  `0.5 * (x_v[t-1] + x_v[t])`.) Is there a difference?

# Multi-objective knapsack

The purpose of this tutorial is to demonstrate how to create and solve a
multi-objective linear program. In addition, it demonstrates how to work with
solvers which return multiple solutions.

## Required packages

This tutorial requires the following packages:

In [ ]:
using JuMP
import HiGHS
import MultiObjectiveAlgorithms as MOA
import Plots
import Test

MultiObjectiveAlgorithms.jl is a package which implements a variety of
algorithms for solving multi-objective optimization problems. Because it is a
long package name, we import it instead as `MOA`.

## Formulation

The [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem) is a
classic problem in mixed-integer programming. Given a collection of items
$i \in I$, each of which has an associated weight, $w_i$, and profit,
$p_i$, the knapsack problem determines which profit-maximizing subset of
items to pack into a knapsack such that the total weight is less than a
capacity $c$. The mathematical formulation is:

$$
\begin{aligned}
\max & \sum\limits_{i \in I} p_i x_i \\
\text{s.t.}\ \ & \sum\limits_{i \in I} w_i x_i \le c\\
& x_i \in \{0, 1\} && \forall i \in I
\end{aligned}
$$
where $x_i$ is $1$ if we pack item $i$ into the knapsack and $0$
otherwise.

For this tutorial, we extend the single-objective knapsack problem by adding
another objective: given a desirability rating, $r_i$, we wish to maximize
the total desirability of the items in our knapsack. Thus, our mathematical
formulation is now:

$$
\begin{aligned}
\max & \sum\limits_{i \in I} p_i x_i \\
     & \sum\limits_{i \in I} r_i x_i \\
\text{s.t.}\ \ & \sum\limits_{i \in I} w_i x_i \le c\\
& x_i \in \{0, 1\} && \forall i \in I
\end{aligned}
$$

## Data

The data for this example was taken from [vOptGeneric](https://github.com/vOptSolver/vOptGeneric.jl),
and the original author was [@xgandibleux](https://github.com/xgandibleux).

In [ ]:
profit = [77, 94, 71, 63, 96, 82, 85, 75, 72, 91, 99, 63, 84, 87, 79, 94, 90]
desire = [65, 90, 90, 77, 95, 84, 70, 94, 66, 92, 74, 97, 60, 60, 65, 97, 93]
weight = [80, 87, 68, 72, 66, 77, 99, 85, 70, 93, 98, 72, 100, 89, 67, 86, 91]
capacity = 900
N = length(profit)

Comparing the capacity to the total weight of all the items:

In [ ]:
capacity / sum(weight)

shows that we can take approximately 64% of the items.

Plotting the items, we see that there are a range of items with different
profits and desirability. Some items have a high profit and a high
desirability, others have a low profit and a high desirability (and vice
versa).

In [ ]:
Plots.scatter(
    profit,
    desire;
    xlabel = "Profit",
    ylabel = "Desire",
    legend = false,
)

The goal of the bi-objective knapsack problem is to choose a subset which
maximizes both objectives.

## JuMP formulation

Our JuMP formulation is a direct translation of the mathematical formulation:

In [ ]:
model = Model()
@variable(model, x[1:N], Bin)
@constraint(model, sum(weight[i] * x[i] for i in 1:N) <= capacity)
@expression(model, profit_expr, sum(profit[i] * x[i] for i in 1:N))
@expression(model, desire_expr, sum(desire[i] * x[i] for i in 1:N))
@objective(model, Max, [profit_expr, desire_expr])

Note how we form a multi-objective program by passing a vector of scalar
objective functions.

## Solution

To solve our model, we need an optimizer which supports multi-objective linear
programs. One option is to use the MultiObjectiveAlgorithms.jl
package.

In [ ]:
set_optimizer(model, () -> MOA.Optimizer(HiGHS.Optimizer))
set_silent(model)

MultiObjectiveAlgorithms.jl supports many different algorithms for solving
multiobjective optimization problems. One option is the epsilon-constraint
method:

In [ ]:
set_attribute(model, MOA.Algorithm(), MOA.EpsilonConstraint())

Let's solve the problem and see the solution

In [ ]:
optimize!(model)
solution_summary(model)

There are 9 solutions available. We can also use `result_count` to see
how many solutions are available:

In [ ]:
result_count(model)

## Accessing multiple solutions

Access the nine different solutions in the model using the `result` keyword to
`solution_summary`, `value`, and `objective_value`:

In [ ]:
solution_summary(model; result = 5)

In [ ]:
objective_value(model; result = 5)

Note that because we set a vector of two objective functions, the objective
value is a vector with two elements. We can also query the value of each
objective separately:

In [ ]:
value(profit_expr; result = 5)

## Visualizing objective space

Unlike single-objective optimization problems, multi-objective optimization
problems do not have a single optimal solution. Instead, the solutions
returned represent possible trade-offs that the decision maker can choose
between the two objectives. A common way to visualize this is by plotting
the objective values of each of the solutions:

In [ ]:
plot = Plots.scatter(
    [value(profit_expr; result = i) for i in 1:result_count(model)],
    [value(desire_expr; result = i) for i in 1:result_count(model)];
    xlabel = "Profit",
    ylabel = "Desire",
    title = "Objective space",
    label = "",
    xlims = (915, 960),
)
for i in 1:result_count(model)
    y = objective_value(model; result = i)
    Plots.annotate!(y[1] - 1, y[2], (i, 10))
end
ideal_point = objective_bound(model)
Plots.scatter!([ideal_point[1]], [ideal_point[2]]; label = "Ideal point")

Visualizing the objective space lets the decision maker choose a solution that
suits their personal preferences. For example, result `#7` is close to the
maximum value of profit, but offers significantly higher desirability compared
with solutions `#8` and `#9`.

The set of items that are chosen in solution `#7` are:

In [ ]:
items_chosen = [i for i in 1:N if value(x[i]; result = 7) > 0.9]

## Next steps

MultiObjectiveAlgorithms.jl implements a number of different
algorithms. Try solving the same problem using `MOA.Dichotomy()`. Does it find
the same solution?

# Column generation

The purpose of this tutorial is to demonstrate the column generation
algorithm. As an example, it solves the [Cutting stock problem](https://en.wikipedia.org/wiki/Cutting_stock_problem).

This tutorial uses the following packages:

In [ ]:
using JuMP
import DataFrames
import HiGHS
import Plots
import SparseArrays

## Background

The cutting stock problem is about cutting large rolls of paper into smaller
pieces.

We denote the set of possible sized pieces that a roll can be cut into by
$i\in 1,\ldots,I$. Each piece $i$ has a width, $w_i$, and a demand,
$d_i$. The width of the large roll is $W$.

Our objective is to minimize the number of rolls needed to meet all demand.

Here's the data that we are going to use in this tutorial:

In [ ]:
struct Piece
    w::Float64
    d::Int
end

struct Data
    pieces::Vector{Piece}
    W::Float64
end

function Base.show(io::IO, d::Data)
    println(io, "Data for the cutting stock problem:")
    println(io, "  W = $(d.W)")
    println(io, "with pieces:")
    println(io, "   i   w_i d_i")
    println(io, "  ------------")
    for (i, p) in enumerate(d.pieces)
        println(io, lpad(i, 4), " ", lpad(p.w, 5), " ", lpad(p.d, 3))
    end
    return
end

function get_data()
    data = [
        75.0 38
        75.0 44
        75.0 30
        75.0 41
        75.0 36
        53.8 33
        53.0 36
        51.0 41
        50.2 35
        32.2 37
        30.8 44
        29.8 49
        20.1 37
        16.2 36
        14.5 42
        11.0 33
        8.6 47
        8.2 35
        6.6 49
        5.1 42
    ]
    return Data([Piece(data[i, 1], data[i, 2]) for i in axes(data, 1)], 100.0)
end

data = get_data()

## Mathematical formulation

To formulate the cutting stock problem as a mixed-integer linear program, we
assume that there is a set of large rolls $j=1,\ldots,J$ to use. Then, we
introduce two classes of decision variables:
* $x_{ij} \ge 0,\; \text{integer}, \; \forall i=1,\ldots,I,\; j=1,\ldots,J$
* $y_{j} \in \{0, 1\},\; \forall j=1,\ldots,J.$
$y_j$ is a binary variable that indicates if we use roll $j$, and
$x_{ij}$ counts how many pieces of size $i$ that we cut from roll $j$.

Our mixed-integer linear program is therefore:
$$
\begin{align}
    \min & \sum\limits_{j=1}^J y_j \\
    \;\;\text{s.t.} & \sum\limits_{i=1}^N w_i x_{ij} \le W y_j & \forall j=1,\ldots,J \\
         & \sum\limits_{j=1}^J x_{ij} \ge d_i & \forall i=1,\ldots,I \\
         & x_{ij} \ge 0 & \forall i=1,\ldots,N, j=1,\ldots,J \\
         & x_{ij} \in \mathbb{Z} & \forall i=1,\ldots,I, j=1,\ldots,J \\
         & y_{j} \in \{0, 1\} & \forall j=1,\ldots,J \\
\end{align}
$$
The objective is to minimize the number of rolls that we use, and the two
constraints ensure that we respect the total width of each large roll and that
we satisfy demand exactly.

The JuMP formulation of this model is:

In [ ]:
I = length(data.pieces)
J = 1_000  # Some large number
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[1:I, 1:J] >= 0, Int)
@variable(model, y[1:J], Bin)
@objective(model, Min, sum(y))
@constraint(model, [i in 1:I], sum(x[i, :]) >= data.pieces[i].d)
@constraint(
    model,
    [j in 1:J],
    sum(data.pieces[i].w * x[i, j] for i in 1:I) <= data.W * y[j],
);

Unfortunately, we can't solve this formulation for realistic instances because
it takes a very long time to solve. (Try removing the time limit.)

In [ ]:
set_time_limit_sec(model, 5.0)
optimize!(model)
solution_summary(model)

However, there is a formulation that solves much faster, and that is to use a
column generation scheme.

## Column generation theory

The key insight for column generation is to recognize that feasible columns
in the $x$ matrix of variables encode _cutting patterns_.

For example, if we look only at the roll $j=1$, then a feasible solution is:

 * $x_{1,1} = 1$ (1 unit of piece \#1)
 * $x_{13,1} = 1$ (1 unit of piece \#13)
 * All other $x_{i,1} = 0$

Another solution is

 * $x_{20,1} = 19$ (19 unit of piece \#20)
 * All other $x_{i,1} = 0$

Cutting patterns like $x_{1,1} = 1$ and $x_{2,1} = 1$ are infeasible
because the combined length is greater than $W$.

Since there are a finite number of ways that we could cut a roll into a
valid cutting pattern, we could create a set of all possible cutting patterns
$p = 1,\ldots,P$, with data $a_{i,p}$ indicating how many units of piece
$i$ we cut in pattern $p$. Then, we can formulate our mixed-integer linear
program as:
$$
\begin{align}
    \min & \sum\limits_{p=1}^P x_p \\
    \;\;\text{s.t.} & \sum\limits_{p=1}^P a_{ip} x_p \ge d_i & \forall i=1,\ldots,I \\
         & x_{p} \ge 0 & \forall p=1,\ldots,P \\
         & x_{p} \in \mathbb{Z} & \forall p=1,\ldots,P
\end{align}
$$

Unfortunately, there will be a very large number of these patterns, so it is
often intractable to enumerate all columns $p=1,\ldots,P$.

Column generation is an iterative algorithm that starts with a small set of
initial patterns, and then cleverly chooses new columns to add to the main
MILP so that we find the optimal solution without having to enumerate every
column.

## Choosing the initial set of patterns

For the initial set of patterns, we create a trivial cutting pattern which
cuts as many units of piece $i$ as will fit.

In [ ]:
patterns = map(1:I) do i
    n_pieces = floor(Int, data.W / data.pieces[i].w)
    return SparseArrays.sparsevec([i], [n_pieces], I)
end

We can visualize the patterns as follows:

In [ ]:
"""
    cutting_locations(data::Data, pattern::SparseArrays.SparseVector)

A function which returns a vector of the locations along the roll at which to
cut in order to produce pattern `pattern`.
"""
function cutting_locations(data::Data, pattern::SparseArrays.SparseVector)
    locations = Float64[]
    offset = 0.0
    for (i, c) in zip(SparseArrays.findnz(pattern)...)
        for _ in 1:c
            offset += data.pieces[i].w
            push!(locations, offset)
        end
    end
    return locations
end

function plot_patterns(data::Data, patterns)
    plot = Plots.bar(;
        xlims = (0, length(patterns) + 1),
        ylims = (0, data.W),
        xlabel = "Pattern",
        ylabel = "Roll length",
    )
    for (i, p) in enumerate(patterns)
        locations = cutting_locations(data, p)
        Plots.bar!(
            plot,
            fill(i, length(locations)),
            reverse(locations);
            bar_width = 0.6,
            label = false,
            color = "#90caf9",
        )
    end
    return plot
end

plot_patterns(data, patterns)

## The base problem

Using the initial set of patterns, we can create and optimize our base model:

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[1:length(patterns)] >= 0, Int)
@objective(model, Min, sum(x))
@constraint(model, demand[i in 1:I], patterns[i]' * x >= data.pieces[i].d)
optimize!(model)
solution_summary(model)

This solution requires 421 rolls. This solution is sub-optimal because the
model does not contain the full set of possible patterns.

How do we find a new column that leads to an improved solution?

## Choosing new columns

Column generation chooses a new column by relaxing the integrality constraint
on $x$ and looking at the dual variable $\pi_i$ associated with demand
constraint $i$.

For example, the dual of `demand[13]` is:

In [ ]:
unset_integer.(x)
optimize!(model)
π_13 = dual(demand[13])

Using the economic interpretation of the dual variable, we can say that a one
unit increase in demand for piece $i$ will cost an extra $\pi_i$ rolls.
Alternatively, we can say that a one unit increase in the left-hand side
(for example, due to a new cutting pattern) will _save_ us $\pi_i$ rolls.
Therefore, we want a new column that maximizes the savings associated with
the dual variables, while respecting the total width of the roll:
$$
\begin{align}
    \max & \sum\limits_{i=1}^I \pi_i y_i \\
    \;\;\text{s.t.} & \sum\limits_{i=1}^I w_i y_{i} \le W \\
         & y_{i} \ge 0 & \forall i=1,\ldots,I \\
         & y_{i} \in \mathbb{Z} & \forall i=1,\ldots,I \\
\end{align}
$$
If this problem, called the _pricing problem_, has an objective value greater
than $1$, then we estimate than adding `y` as the coefficients of a new
column will decrease the objective by more than the cost of an extra roll.

Here is code to solve the pricing problem:

In [ ]:
function solve_pricing(data::Data, π::Vector{Float64})
    I = length(π)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, y[1:I] >= 0, Int)
    @constraint(model, sum(data.pieces[i].w * y[i] for i in 1:I) <= data.W)
    @objective(model, Max, sum(π[i] * y[i] for i in 1:I))
    optimize!(model)
    number_of_rolls_saved = objective_value(model)
    if number_of_rolls_saved > 1 + 1e-8
        # Benefit of pattern is more than the cost of a new roll plus some
        # tolerance
        return SparseArrays.sparse(round.(Int, value.(y)))
    end
    return nothing
end

If we solve the pricing problem with an artificial dual vector:

In [ ]:
solve_pricing(data, [1.0 / i for i in 1:I])

the solution is a roll with 1 unit of piece \#1, 1 unit of piece \#17, and 3
units of piece \#20.

If we solve the pricing problem with a dual vector of zeros, then the benefit
of the new pattern is less than the cost of a roll, and so the function
returns `nothing`:

In [ ]:
solve_pricing(data, zeros(I))

## Iterative algorithm

Now we can combine our base model with the pricing subproblem in an iterative
column generation scheme:

In [ ]:
while true
    # Solve the linear relaxation
    optimize!(model)
    # Obtain a new dual vector
    π = dual.(demand)
    # Solve the pricing problem
    new_pattern = solve_pricing(data, π)
    # Stop iterating if there is no new pattern
    if new_pattern === nothing
        @info "No new patterns, terminating the algorithm."
        break
    end
    push!(patterns, new_pattern)
    # Create a new column
    push!(x, @variable(model, lower_bound = 0))
    # Update the objective coefficient of the new column
    set_objective_coefficient(model, x[end], 1.0)
    # Update the non-zeros in the coefficient matrix
    for (i, count) in zip(SparseArrays.findnz(new_pattern)...)
        set_normalized_coefficient(demand[i], x[end], count)
    end
    println("Found new pattern. Total patterns = $(length(patterns))")
end

We found lots of new patterns. Here's pattern 21:

In [ ]:
patterns[21]

Let's have a look at the patterns now:

In [ ]:
plot_patterns(data, patterns)

## Looking at the solution

Let's see how many of each column we need:

In [ ]:
solution = DataFrames.DataFrame([
    (pattern = p, rolls = value(x_p)) for (p, x_p) in enumerate(x)
])
filter!(row -> row.rolls > 0, solution)

Since we solved a linear program, some of our columns have fractional
solutions. We can create a integer feasible solution by rounding up the
orders. This requires 341 rolls:

In [ ]:
sum(ceil.(Int, solution.rolls))

Alternatively, we can re-introduce the integrality constraints and resolve the
problem:

In [ ]:
set_integer.(x)
optimize!(model)
solution = DataFrames.DataFrame([
    (pattern = p, rolls = value(x_p)) for (p, x_p) in enumerate(x)
])
filter!(row -> row.rolls > 0, solution)

This now requires 334 rolls:

In [ ]:
sum(solution.rolls)

Note that this may not be the global minimum because we are not adding new
columns during the solution of the mixed-integer problem `model` (an algorithm
known as [branch and price](https://en.wikipedia.org/wiki/Branch_and_price)).
Nevertheless, the column generation algorithm typically finds good integer
feasible solutions to an otherwise intractable optimization problem.

## Next steps

* Our objective function is to minimize the total number of rolls. What is the
  total length of waste? How does that compare to the total demand?
* Writing the optimization algorithm is only part of the challenge. Can you
  develop a better way to communicate the solution to stakeholders?

# Design patterns for larger models

JuMP makes it easy to build and solve optimization models. However, once you
start to construct larger models, and especially ones that interact with
external data sources or have customizable sets of variables and constraints
based on client choices, you may find that your scripts become unwieldy. This
tutorial demonstrates a variety of ways in which you can structure larger JuMP
models to improve their readability and maintainability.

**Tip**
    This tutorial is more advanced than the other "Getting started" tutorials.
    It's in the "Getting started" section to give you an early preview of how
    JuMP makes it easy to structure larger models. However, if you are new to
    JuMP you may want to briefly skim the tutorial, and come back to it once
    you have written a few JuMP models.

## Overview

This tutorial uses explanation-by-example. We're going to start with a simple
[knapsack model](https://en.wikipedia.org/wiki/Knapsack_problem), and then
expand it to add various features and structure.

## A simple script

Your first prototype of a JuMP model is probably a script that uses a small
set of hard-coded data.

In [ ]:
using JuMP, HiGHS
profit = [5, 3, 2, 7, 4]
weight = [2, 8, 4, 2, 5]
capacity = 10
N = 5
model = Model(HiGHS.Optimizer)
@variable(model, x[1:N], Bin)
@objective(model, Max, sum(profit[i] * x[i] for i in 1:N))
@constraint(model, sum(weight[i] * x[i] for i in 1:N) <= capacity)
optimize!(model)
value.(x)

The benefits of this approach are:
 * it is quick to code
 * it is quick to make changes.

The downsides include:
 * all variables are global (read [Performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/))
 * it is easy to introduce errors, for example, having `profit` and `weight` be
   vectors of different lengths, or not match `N`
 * the solution, `x[i]`, is hard to interpret without knowing the order in
   which we provided the data.

## Wrap the model in a function

A good next step is to wrap your model in a function. This is useful for a few
reasons:
 * it removes global variables
 * it encapsulates the JuMP model and forces you to clarify your inputs and
   outputs
 * we can add some error checking.

In [ ]:
function solve_knapsack_1(profit::Vector, weight::Vector, capacity::Real)
    if length(profit) != length(weight)
        throw(DimensionMismatch("profit and weight are different sizes"))
    end
    N = length(weight)
    model = Model(HiGHS.Optimizer)
    @variable(model, x[1:N], Bin)
    @objective(model, Max, sum(profit[i] * x[i] for i in 1:N))
    @constraint(model, sum(weight[i] * x[i] for i in 1:N) <= capacity)
    optimize!(model)
    return value.(x)
end

solve_knapsack_1([5, 3, 2, 7, 4], [2, 8, 4, 2, 5], 10)

## Create better data structures

Although we can check for errors like mis-matched vector lengths, if you start
to develop models with a lot of data, keeping track of vectors and lengths and
indices is fragile and a common source of bugs. A good solution is to use
Julia's type system to create an abstraction over your data.

For example, we can create a `struct` that represents a single object, with a
constructor that lets us validate assumptions on the input data:

In [ ]:
struct KnapsackObject
    profit::Float64
    weight::Float64
    function KnapsackObject(profit::Float64, weight::Float64)
        if weight < 0
            throw(DomainError("Weight of object cannot be negative"))
        end
        return new(profit, weight)
    end
end

as well as a `struct` that holds a dictionary of objects and the knapsack's
capacity:

In [ ]:
struct KnapsackData
    objects::Dict{String,KnapsackObject}
    capacity::Float64
end

Here's what our data might look like now:

In [ ]:
objects = Dict(
    "apple" => KnapsackObject(5.0, 2.0),
    "banana" => KnapsackObject(3.0, 8.0),
    "cherry" => KnapsackObject(2.0, 4.0),
    "date" => KnapsackObject(7.0, 2.0),
    "eggplant" => KnapsackObject(4.0, 5.0),
)
data = KnapsackData(objects, 10.0)

If you want, you can add custom printing to make it easier to visualize:

In [ ]:
function Base.show(io::IO, data::KnapsackData)
    println(io, "A knapsack with capacity $(data.capacity) and possible items:")
    for (k, v) in data.objects
        println(
            io,
            "  $(rpad(k, 8)) : profit = $(v.profit), weight = $(v.weight)",
        )
    end
    return
end

data

Then, we can re-write our `solve_knapsack` function to take our `KnapsackData`
as input:

In [ ]:
function solve_knapsack_2(data::KnapsackData)
    model = Model(HiGHS.Optimizer)
    @variable(model, x[keys(data.objects)], Bin)
    @objective(model, Max, sum(v.profit * x[k] for (k, v) in data.objects))
    @constraint(
        model,
        sum(v.weight * x[k] for (k, v) in data.objects) <= data.capacity,
    )
    optimize!(model)
    return value.(x)
end

solve_knapsack_2(data)

## Read in data from files

Having a data structure is a good step. But it is still annoying that we have
to hard-code the data into Julia. A good next step is to separate the data
into an external file format; JSON is a common choice.

In [ ]:
json_data = """
{
    "objects": {
        "apple": {"profit": 5.0, "weight": 2.0},
        "banana": {"profit": 3.0, "weight": 8.0},
        "cherry": {"profit": 2.0, "weight": 4.0},
        "date": {"profit": 7.0, "weight": 2.0},
        "eggplant": {"profit": 4.0, "weight": 5.0}
    },
    "capacity": 10.0
}
"""
temp_dir = mktempdir()
knapsack_json_filename = joinpath(temp_dir, "knapsack.json")
# Instead of writing a new file here you could replace `knapsack_json_filename`
# with the path to a local file.
write(knapsack_json_filename, json_data);

Now let's write a function that reads this file and builds a `KnapsackData`
object:

In [ ]:
import JSON

function read_data(filename)
    d = JSON.parsefile(filename)
    return KnapsackData(
        Dict(
            k => KnapsackObject(v["profit"], v["weight"]) for
            (k, v) in d["objects"]
        ),
        d["capacity"],
    )
end

data = read_data(knapsack_json_filename)

## Add options via if-else

At this point, we have data in a file format which we can load and solve a
single problem. For many users, this might be sufficient. However, at some
point you may be asked to add features like "but what if we want to take more
than one of a particular item?"

If this is the first time that you've been asked to add a feature, adding
options via `if-else` statements is a good approach. For example, we might
write:

In [ ]:
function solve_knapsack_3(data::KnapsackData; binary_knapsack::Bool)
    model = Model(HiGHS.Optimizer)
    if binary_knapsack
        @variable(model, x[keys(data.objects)], Bin)
    else
        @variable(model, x[keys(data.objects)] >= 0, Int)
    end
    @objective(model, Max, sum(v.profit * x[k] for (k, v) in data.objects))
    @constraint(
        model,
        sum(v.weight * x[k] for (k, v) in data.objects) <= data.capacity,
    )
    optimize!(model)
    return value.(x)
end

Now we can solve the binary knapsack:

In [ ]:
solve_knapsack_3(data; binary_knapsack = true)

And an integer knapsack where we can take more than one copy of each item:

In [ ]:
solve_knapsack_3(data; binary_knapsack = false)

## Add configuration options via dispatch

If you get repeated requests to add different options, you'll quickly find
yourself in a mess of different flags and `if-else` statements. It's hard to
write, hard to read, and hard to ensure you haven't introduced any bugs.
A good solution is to use Julia's type dispatch to control the configuration
of the model. The easiest way to explain this is by example.

First, start by defining a new abstract type, as well as new subtypes for each
of our options. These types are going to control the configuration of the
knapsack model.

In [ ]:
abstract type AbstractConfiguration end

struct BinaryKnapsackConfig <: AbstractConfiguration end

struct IntegerKnapsackConfig <: AbstractConfiguration end

Then, we rewrite our `solve_knapsack` function to take a `config` argument,
and we introduce an `add_knapsack_variables` function to abstract the creation
of our variables.

In [ ]:
function solve_knapsack_4(data::KnapsackData, config::AbstractConfiguration)
    model = Model(HiGHS.Optimizer)
    x = add_knapsack_variables(model, data, config)
    @objective(model, Max, sum(v.profit * x[k] for (k, v) in data.objects))
    @constraint(
        model,
        sum(v.weight * x[k] for (k, v) in data.objects) <= data.capacity,
    )
    optimize!(model)
    return value.(x)
end

For the binary knapsack problem, `add_knapsack_variables` looks like this:

In [ ]:
function add_knapsack_variables(
    model::Model,
    data::KnapsackData,
    ::BinaryKnapsackConfig,
)
    return @variable(model, x[keys(data.objects)], Bin)
end

For the integer knapsack problem, `add_knapsack_variables` looks like this:

In [ ]:
function add_knapsack_variables(
    model::Model,
    data::KnapsackData,
    ::IntegerKnapsackConfig,
)
    return @variable(model, x[keys(data.objects)] >= 0, Int)
end

Now we can solve the binary knapsack:

In [ ]:
solve_knapsack_4(data, BinaryKnapsackConfig())

and the integer knapsack problem:

In [ ]:
solve_knapsack_4(data, IntegerKnapsackConfig())

The main benefit of the dispatch approach is that you can quickly add new
options without needing to modify the existing code. For example:

In [ ]:
struct UpperBoundedKnapsackConfig <: AbstractConfiguration
    limit::Int
end

function add_knapsack_variables(
    model::Model,
    data::KnapsackData,
    config::UpperBoundedKnapsackConfig,
)
    return @variable(model, 0 <= x[keys(data.objects)] <= config.limit, Int)
end

solve_knapsack_4(data, UpperBoundedKnapsackConfig(3))

## Generalize constraints and objectives

It's easy to extend the dispatch approach to constraints and objectives as
well. The key points to notice in the next two functions are that:
 * we can access registered variables via `model[:x]`
 * we can define generic functions which accept any `AbstractConfiguration` as a
   configuration argument. That means we can implement a single method and
   have it apply to multiple configuration types.

In [ ]:
function add_knapsack_constraints(
    model::Model,
    data::KnapsackData,
    ::AbstractConfiguration,
)
    x = model[:x]
    @constraint(
        model,
        capacity_constraint,
        sum(v.weight * x[k] for (k, v) in data.objects) <= data.capacity,
    )
    return
end

function add_knapsack_objective(
    model::Model,
    data::KnapsackData,
    ::AbstractConfiguration,
)
    x = model[:x]
    @objective(model, Max, sum(v.profit * x[k] for (k, v) in data.objects))
    return
end

function solve_knapsack_5(data::KnapsackData, config::AbstractConfiguration)
    model = Model(HiGHS.Optimizer)
    add_knapsack_variables(model, data, config)
    add_knapsack_constraints(model, data, config)
    add_knapsack_objective(model, data, config)
    optimize!(model)
    return value.(model[:x])
end

solve_knapsack_5(data, BinaryKnapsackConfig())

## Remove solver dependence, add error checks

Compared to where we started, our knapsack model is now significantly
different. We've wrapped it in a function, defined some data types, and
introduced configuration options to control the variables and constraints that
get added. There are a few other steps we can do to further improve things:
 * remove the dependence on `HiGHS`
 * add checks that we found an optimal solution
 * add a helper function to avoid the need to explicitly construct the data.

In [ ]:
function solve_knapsack_6(
    optimizer,
    data::KnapsackData,
    config::AbstractConfiguration,
)
    model = Model(optimizer)
    add_knapsack_variables(model, data, config)
    add_knapsack_constraints(model, data, config)
    add_knapsack_objective(model, data, config)
    optimize!(model)
    if termination_status(model) != OPTIMAL
        @warn("Model not solved to optimality")
        return nothing
    end
    return value.(model[:x])
end

function solve_knapsack_6(
    optimizer,
    data::String,
    config::AbstractConfiguration,
)
    return solve_knapsack_6(optimizer, read_data(data), config)
end

solution = solve_knapsack_6(
    HiGHS.Optimizer,
    knapsack_json_filename,
    BinaryKnapsackConfig(),
)

## Create a module

Now we're ready to expose our model to the wider world. That might be as part
of a larger Julia project that we're contributing to, or as a stand-alone
script that we can run on-demand. In either case, it's good practice to wrap
everything in a module. This further encapsulates our code into a single
namespace, and we can add documentation in the form of
[docstrings](https://docs.julialang.org/en/v1/manual/documentation/).

Some good rules to follow when creating a module are:
* use `import` in a module instead of `using` to make it clear which functions
  are from which packages
* use `_` to start function and type names that are considered private
* add docstrings to all public variables and functions.

In [ ]:
module KnapsackModel

import JuMP
import JSON

struct _KnapsackObject
    profit::Float64
    weight::Float64
    function _KnapsackObject(profit::Float64, weight::Float64)
        if weight < 0
            throw(DomainError("Weight of object cannot be negative"))
        end
        return new(profit, weight)
    end
end

struct _KnapsackData
    objects::Dict{String,_KnapsackObject}
    capacity::Float64
end

function _read_data(filename)
    d = JSON.parsefile(filename)
    return _KnapsackData(
        Dict(
            k => _KnapsackObject(v["profit"], v["weight"]) for
            (k, v) in d["objects"]
        ),
        d["capacity"],
    )
end

abstract type _AbstractConfiguration end

"""
    BinaryKnapsackConfig()

Create a binary knapsack problem where each object can be taken 0 or 1 times.
"""
struct BinaryKnapsackConfig <: _AbstractConfiguration end

"""
    IntegerKnapsackConfig()

Create an integer knapsack problem where each object can be taken any number of
times.
"""
struct IntegerKnapsackConfig <: _AbstractConfiguration end

function _add_knapsack_variables(
    model::JuMP.Model,
    data::_KnapsackData,
    ::BinaryKnapsackConfig,
)
    return JuMP.@variable(model, x[keys(data.objects)], Bin)
end

function _add_knapsack_variables(
    model::JuMP.Model,
    data::_KnapsackData,
    ::IntegerKnapsackConfig,
)
    return JuMP.@variable(model, x[keys(data.objects)] >= 0, Int)
end

function _add_knapsack_constraints(
    model::JuMP.Model,
    data::_KnapsackData,
    ::_AbstractConfiguration,
)
    x = model[:x]
    JuMP.@constraint(
        model,
        capacity_constraint,
        sum(v.weight * x[k] for (k, v) in data.objects) <= data.capacity,
    )
    return
end

function _add_knapsack_objective(
    model::JuMP.Model,
    data::_KnapsackData,
    ::_AbstractConfiguration,
)
    x = model[:x]
    JuMP.@objective(model, Max, sum(v.profit * x[k] for (k, v) in data.objects))
    return
end

function _solve_knapsack(
    optimizer,
    data::_KnapsackData,
    config::_AbstractConfiguration,
)
    model = JuMP.Model(optimizer)
    _add_knapsack_variables(model, data, config)
    _add_knapsack_constraints(model, data, config)
    _add_knapsack_objective(model, data, config)
    JuMP.optimize!(model)
    if JuMP.termination_status(model) != JuMP.OPTIMAL
        @warn("Model not solved to optimality")
        return nothing
    end
    return JuMP.value.(model[:x])
end

"""
    solve_knapsack(
        optimizer,
        knapsack_json_filename::String,
        config::_AbstractConfiguration,
    )

Solve the knapsack problem and return the optimal primal solution

# Arguments

 * `optimizer` : an object that can be passed to `JuMP.Model` to construct a new
   JuMP model.
 * `knapsack_json_filename` : the filename of a JSON file containing the data for the
   problem.
 * `config` : an object to control the type of knapsack model constructed.
   Valid options are:
    * `BinaryKnapsackConfig()`
    * `IntegerKnapsackConfig()`

# Returns

 * If an optimal solution exists: a `JuMP.DenseAxisArray` that maps the `String`
   name of each object to the number of objects to pack into the knapsack.
 * Otherwise, `nothing`, indicating that the problem does not have an optimal
   solution.

# Examples

```julia
solution = solve_knapsack(
    HiGHS.Optimizer,
    "path/to/data.json",
    BinaryKnapsackConfig(),
)
```

```julia
solution = solve_knapsack(
    MOI.OptimizerWithAttributes(HiGHS.Optimizer, "output_flag" => false),
    "path/to/data.json",
    IntegerKnapsackConfig(),
)
```
"""
function solve_knapsack(
    optimizer,
    knapsack_json_filename::String,
    config::_AbstractConfiguration,
)
    data = _read_data(knapsack_json_filename)
    return _solve_knapsack(optimizer, data, config)
end

end

Finally, you can call your model:

In [ ]:
import .KnapsackModel

KnapsackModel.solve_knapsack(
    HiGHS.Optimizer,
    knapsack_json_filename,
    KnapsackModel.BinaryKnapsackConfig(),
)

**Note**
    The `.` in `.KnapsackModel` denotes that it is a submodule and not a
    separate package that we installed with `Pkg.add`. If you put the
    `KnapsackModel` in a separate file, load it with:
    ```julia
    include("path/to/KnapsackModel.jl")
    import .KnapsackModel
    ```

## Add tests

As a final step, you should add tests for your model. This often means testing
on a small problem for which you can work out the optimal solution by hand.
The Julia standard library `Test` has good unit-testing functionality.

In [ ]:
import .KnapsackModel
using Test

@testset "KnapsackModel" begin
    @testset "feasible_binary_knapsack" begin
        x = KnapsackModel.solve_knapsack(
            HiGHS.Optimizer,
            knapsack_json_filename,
            KnapsackModel.BinaryKnapsackConfig(),
        )
        @test isapprox(x["apple"], 1, atol = 1e-5)
        @test isapprox(x["banana"], 0, atol = 1e-5)
        @test isapprox(x["cherry"], 0, atol = 1e-5)
        @test isapprox(x["date"], 1, atol = 1e-5)
        @test isapprox(x["eggplant"], 1, atol = 1e-5)
    end
    @testset "feasible_integer_knapsack" begin
        x = KnapsackModel.solve_knapsack(
            HiGHS.Optimizer,
            knapsack_json_filename,
            KnapsackModel.IntegerKnapsackConfig(),
        )
        @test isapprox(x["apple"], 0, atol = 1e-5)
        @test isapprox(x["banana"], 0, atol = 1e-5)
        @test isapprox(x["cherry"], 0, atol = 1e-5)
        @test isapprox(x["date"], 5, atol = 1e-5)
        @test isapprox(x["eggplant"], 0, atol = 1e-5)
    end
    @testset "infeasible_binary_knapsack" begin
        dir = mktempdir()
        infeasible_filename = joinpath(dir, "infeasible.json")
        write(
            infeasible_filename,
            """{
                "objects": {
                    "apple": {"profit": 5.0, "weight": 2.0},
                    "banana": {"profit": 3.0, "weight": 8.0},
                    "cherry": {"profit": 2.0, "weight": 4.0},
                    "date": {"profit": 7.0, "weight": 2.0},
                    "eggplant": {"profit": 4.0, "weight": 5.0}
                },
                "capacity": -10.0
            }""",
        )
        x = KnapsackModel.solve_knapsack(
            HiGHS.Optimizer,
            infeasible_filename,
            KnapsackModel.BinaryKnapsackConfig(),
        )
        @test x === nothing
    end
end

**Tip**
    Place these tests in a separate file `test_knapsack_model.jl` so that you
    can run the tests by adding `include("test_knapsack_model.jl")` to any
    file where needed.

## Next steps

We've only briefly scratched the surface of ways to create and structure large
JuMP models, so consider this tutorial a starting point, rather than a
comprehensive list of all the possible ways to structure JuMP models.  If you
are embarking on a large project that uses JuMP, a good next step is to
look at ways people have written large JuMP projects "in the wild."

Here are some good examples (all co-incidentally related to energy):
* AnyMOD.jl
  * [JuMP-dev 2021 talk](https://www.youtube.com/watch?v=QE_tNDER0F4)
  * [source code](https://github.com/leonardgoeke/AnyMOD.jl)
* PowerModels.jl
  * [JuMP-dev 2021 talk](https://www.youtube.com/watch?v=POOt1FCA8LI)
  * [source code](https://github.com/lanl-ansi/PowerModels.jl)
* PowerSimulations.jl
   * [JuliaCon 2021 talk](https://www.youtube.com/watch?v=-ZoO3npjwYU)
   * [source code](https://github.com/NREL-SIIP/PowerSimulations.jl)
* UnitCommitment.jl
  * [JuMP-dev 2021 talk](https://www.youtube.com/watch?v=rYUZK9kYeIY)
  * [source code](https://github.com/ANL-CEEESA/UnitCommitment.jl)